In [1]:
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 25.0 MB/s 
     |████████████████████████████████| 588.3 MB 6.1 kB/s 
     |████████████████████████████████| 6.0 MB 51.4 MB/s 
     |████████████████████████████████| 439 kB 68.0 MB/s 
     |████████████████████████████████| 1.7 MB 52.3 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1

In [2]:
#importing the libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import tensorflow_text as text
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [3]:
with open('Intent.json') as chatfile:
    data = json.load(chatfile)

In [4]:
tags = []
inputs = []
responses = {}

In [5]:
for intention in data['intents']:
    responses[intention['intent']] = intention['responses']
    for lines in intention['text']:
        inputs.append(lines)
        tags.append(intention['intent'])


In [6]:
data = pd.DataFrame({"inputs":inputs,"tags":tags})

In [7]:
data

,inputs,tags
0,Hi,Greeting
1,Hi there,Greeting
2,Hola,Greeting
3,Hello,Greeting
4,Hello there,Greeting
...,...,...
63,mehnn,confused
64,sha let's not get into it,confused
65,Everyhting okay??,hidden
66,Are you good??,hidden


In [8]:
classes = len(data.tags.unique())
class_names = data.tags.unique()

In [22]:
class_names

array(['Greeting', 'CourtesyGreeting', 'NotTalking2U', 'Shutup',
       'Swearing', 'GoodBye', 'CourtesyGoodBye', 'WhoAmI', 'Clever',
       'small_talk', 'confused', 'hidden'], dtype=object)

In [11]:
X = data.inputs
y = le.fit_transform(data.tags)

x_train,x_test,y_train,y_test = train_test_split(X,y, test_size = 0.1)

In [12]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [13]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.84351724, -0.51327276, -0.88845724, ..., -0.74748844,
        -0.7531474 ,  0.91964495],
       [-0.87208337, -0.5054394 , -0.94446665, ..., -0.85847497,
        -0.71745324,  0.8808295 ]], dtype=float32)>

In [14]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
x = Dense(12,activation="softmax")(outputs['pooled_output'])
model = tf.keras.Model(inputs=[text_input], outputs = [x])
#compiling the model
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
#training the model
train = model.fit(x_train,y_train,epochs=200)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Epoch 1/200
2/2 [==============================] - 9s 274ms/step - loss: 2.5518 - accuracy: 0.0492
Epoch 2/200
2/2 [==============================] - 1s 272ms/step - loss: 2.3538 - accuracy: 0.2131
Epoch 3/200
2/2 [==============================] - 1s 276ms/step - loss: 2.2932 - accuracy: 0.1967
Epoch 4/200
2/2 [==============================] - 1s 275ms/step - loss: 2.2583 - accuracy: 0.1967
Epoch 5/200
2/2 [==============================] - 1s 279ms/step - loss: 2.1945 - accuracy: 0.2131
Epoch 6/200
2/2 [==============================] - 1s 263ms/step - loss: 2.1546 - accuracy: 0.2787
Epoch 7/200
2/2 [==============================] - 1s 277ms/step - loss: 2.1077 - accuracy: 0.3115
Epoch 8/200
2/2 [==============================] - 1s 273ms/step - loss: 2.0785 - accuracy: 0.3443
Epoch 9/200
2/2 [==============================] - 1s 278ms/step - loss: 2.0468 - accuracy: 0.3443
Epoch 10/200
2/2 [==============================] - 1s 273ms/step - loss: 2.0118 - accuracy: 0.3279
Epoch 11/

In [16]:
model.save('LISA')

In [ ]:
import random
while True:
  texts_p = []
  prediction_input = input('You : ')

  output = model.predict([prediction_input])
  output = output.argmax()
  #finding the right tag and predicting[
  response_tag = le.inverse_transform([output])[0]
  print("Going Merry : ",random.choice(responses[response_tag]))
  if output == "goodbye":
      break